# Analýza věku a pohlaví podle pětiletých věkových skupin

### Import knihoven


In [1]:
import pandas as pd
import numpy as np
import pymongo

### Poznámky

In [ ]:
# https://www.czso.cz/csu/rso/uzemne_technicke_jednotky
# https://scitani.gov.cz/vysledky
# https://www.czso.cz/csu/czso/vysledky-scitani-2021-otevrena-data

### Pomocné funkce

In [36]:
# Tisk čísla s oddělovači čárek
def format_with_commas(number):
    return "{:,}".format(number)

# Hezký tisk slovníku
def print_dictionary_ages(text, dict):
    print(f"{text}")
    for key, value in dict.items():
        print(f"\t=> {key}: {value} ({(value / analysis_result['count'])*100:.2f}%)")

### Načtení dat do paměti z MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["scitani_lidu"]
collection = db["obyvatelstvo_pohlavi_vek_5"]

data = []

for index, document in enumerate(collection.find()):
    data.append(document)

data = pd.DataFrame(data)
data = data.replace({np.nan: None})

### Analýza

In [19]:
# Pomocné CIS kódy
cis_vek = {
    1100000004: '0 - 4 roky',
    1300050009: '5 - 9 let',
    1300100014: '10 - 14 let',
    1300150019: '15 - 19 let',
    1300200024: '20 - 24 let',
    1300250029: '25 - 29 let',
    1300300034: '30 - 34 let',
    1300350039: '35 - 39 let',
    1300400044: '40 - 44 let',
    1300450049: '45 - 49 let',
    1300500054: '50 - 54 let',
    1300550059: '55 - 59 let',
    1300600064: '60 - 64 let',
    1300650069: '65 - 69 let',
    1300700074: '70 - 74 let',
    1300750079: '75 - 79 let',
    1300800084: '80 - 84 let',
    1300850089: '85 - 89 let',
    1300900094: '90 - 94 let',
    1300950099: '95 - 99 let',
    1201009999: '100 a více let'
}

In [28]:
analysis_result = dict()

# Celkový počet obyvatel
analysis_result['count'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle pohlaví
analysis_result['count_women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 2)]['hodnota'])
analysis_result['count_men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věku
analysis_result['count_ages'] = {}
for key, value in cis_vek.items():
    analysis_result['count_ages'][value] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_txt'].isnull())]['hodnota'])

### Výsledky analýzy

In [37]:
# Celkový počet obyvatel
print(f"Celkový počet obyvatel: {format_with_commas(analysis_result['count'])}")
print(f"Celkový počet žen: {format_with_commas(analysis_result['count_women'])} ({(analysis_result['count_women'] / analysis_result['count'])*100:.2f}%)")
print(f"Celkový počet muž: {format_with_commas(analysis_result['count_men'])} ({(analysis_result['count_men'] / analysis_result['count'])*100:.2f}%)")
print_dictionary_ages("Celkový počet obyvatel podle věku:", analysis_result['count_ages'])

Celkový počet obyvatel: 10,524,167
Celkový počet žen: 5,337,619 (50.72%)
Celkový počet muž: 5,186,548 (49.28%)
Celkový počet obyvatel podle věku:
	=> 0 - 4 roky: 560632 (5.33%)
	=> 5 - 9 let: 550024 (5.23%)
	=> 10 - 14 let: 581104 (5.52%)
	=> 15 - 19 let: 489836 (4.65%)
	=> 20 - 24 let: 477063 (4.53%)
	=> 25 - 29 let: 601168 (5.71%)
	=> 30 - 34 let: 690561 (6.56%)
	=> 35 - 39 let: 719089 (6.83%)
	=> 40 - 44 let: 857917 (8.15%)
	=> 45 - 49 let: 877616 (8.34%)
	=> 50 - 54 let: 688110 (6.54%)
	=> 55 - 59 let: 666391 (6.33%)
	=> 60 - 64 let: 616608 (5.86%)
	=> 65 - 69 let: 668081 (6.35%)
	=> 70 - 74 let: 618629 (5.88%)
	=> 75 - 79 let: 419368 (3.98%)
	=> 80 - 84 let: 243373 (2.31%)
	=> 85 - 89 let: 135311 (1.29%)
	=> 90 - 94 let: 52786 (0.50%)
	=> 95 - 99 let: 9853 (0.09%)
	=> 100 a více let: 647 (0.01%)
